In [ ]:
import tensorflow as tf### models
import pandas as pd ### reading and processing data
import seaborn as sns ### visualization
import numpy as np### math computations
import matplotlib.pyplot as plt### plotting bar chart
from tensorflow.keras.layers import Normalization, Dense, InputLayer
from tensorflow.keras.losses import MeanSquaredError, Huber, MeanAbsoluteError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam

In [ ]:
!pip install opendatasets


In [ ]:
import opendatasets as od
import os

In [ ]:
od.download("https://www.kaggle.com/datasets/mayankpatel14/second-hand-used-cars-data-set-linear-regression")

In [ ]:
os.listdir("second-hand-used-cars-data-set-linear-regression")

In [ ]:
train_df = pd.read_csv("second-hand-used-cars-data-set-linear-regression/train.csv")

In [ ]:
train_df

In [ ]:
train_df.shape

In [ ]:
sns.pairplot(train_df[['v.id', 'on road old', 'on road now', 'years', 'km', 'rating', 'condition', 'economy', 'top speed', 'hp', 'torque', 'current price']])

In [ ]:
tensor_data = tf.constant(train_df)
tensor_data = tf.cast(tensor_data, tf.float32)

print(tensor_data)

In [ ]:
tensor_data = tf.random.shuffle(tensor_data)
print(tensor_data)

In [ ]:
tensor_data.shape

In [ ]:
X = tensor_data[:,3:-1]
print(X.shape)

In [ ]:
Y = tensor_data[:, -1]
print(Y[:5].shape)
Y = tf.expand_dims(Y, axis = -1)
print(Y[:5])

In [ ]:
normalizer = Normalization(axis = -1, mean= 5, variance = 4)
x_normalized = tf.constant([[3,4,5,6,7,8],
                            [4,5,6,7,8,9]])
normalizer(x_normalized)

In [ ]:
normalizer = Normalization()
normalizer.adapt(X)
normalizer(X)[:5]

In [ ]:
Train_Ratio = 0.8
Val_Ratio = 0.1
Test_Ratio = 0.1
Dataset_Size = len(X)

In [ ]:
x_train = X[:int(Dataset_Size * Train_Ratio)]
y_train = Y[:int(Dataset_Size * Train_Ratio)]

print(x_train.shape)
print(y_train.shape)

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size = 8, reshuffle_each_iteration = True).batch(32).prefetch(tf.data.AUTOTUNE)

In [ ]:
train_dataset

In [ ]:
print(train_dataset)

In [ ]:
x_val = X[int(Dataset_Size * Train_Ratio): int(Dataset_Size * (Train_Ratio + Val_Ratio))]
y_val = Y[int(Dataset_Size * Train_Ratio): int(Dataset_Size * (Train_Ratio + Val_Ratio))]



In [ ]:
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = val_dataset.shuffle(buffer_size = 8, reshuffle_each_iteration = True).batch(32).prefetch(tf.data.AUTOTUNE)

In [ ]:
x_test = X[int(Dataset_Size*(Train_Ratio + Val_Ratio)):]
y_test = Y[int(Dataset_Size* (Train_Ratio+ Val_Ratio)):]


In [ ]:
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_dataset = train_dataset.shuffle(buffer_size = 8, reshuffle_each_iteration = True).batch(32).prefetch(tf.data.AUTOTUNE)

In [ ]:
normalizer = Normalization()
normalizer.adapt(x_train)
normalizer(x_train)[:5]

In [ ]:
model = tf.keras.Sequential([
    InputLayer(input_shape = (8,)),
    normalizer,
    Dense(128, activation = "relu"),
    Dense(128, activation = "relu"),
    Dense(128, activation="relu"),
    Dense(1),
])

model.summary()

In [ ]:
tf.keras.utils.plot_model(model, to_file = "model.png", show_shapes = True)


In [ ]:
model.compile(optimizer = Adam(learning_rate = 0.1),
              loss = MeanAbsoluteError(),
              metrics = [MeanAbsoluteError(), MeanSquaredError(), tf.keras.metrics.RootMeanSquaredError()])

In [ ]:
model.compile(optimizer = Adam(learning_rate = 0.1),
              loss = MeanAbsoluteError(),
              metrics = [RootMeanSquaredError()])

In [ ]:
history = model.fit(train_dataset, validation_data = val_dataset, epochs = 100, verbose = 1)

In [ ]:
plt.plot(history.history["loss"])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel("loss")
plt.xlabel('epoch')
plt.legend(['train', 'val_loss'])
plt.show()

In [ ]:
plt.plot(history.history['root_mean_squared_error'])
plt.plot(history.history['val_root_mean_squared_error'])
plt.title('model performance')
plt.ylabel('rmse')
plt.xlabel('epoch')
plt.legend(['train', 'val'])
plt.show()